## CUCM Certificate Cleanup

### Process Overview
1. Collect all the trust certificates from CUCM using admin CLI
2. Inspect each certificate for validity
3. Delete the expired certificates using the admin CLI
4. Restart the tomcat service

In [ ]:
import time
import sys
import itertools
import paramiko
from paramiko_expect import SSHClientInteraction
import os
import os.path
import re
import pandas as pd
pd.set_option('display.max_colwidth', None)
from datetime import timezone
from email.utils import parsedate_to_datetime
from email.utils import localtime
from dotenv import load_dotenv

### 1. Collect all the trust certificates from CUCM using admin CLI

In [ ]:
# load all environment variables
load_dotenv()

#Create a SSH connection to the CUCM
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

hostname = os.environ.get("HOSTNAME")
user = os.environ.get("USERNAME")
secret = os.environ.get("PASSWORD")

ssh.connect(hostname, username=user, password=secret)
interact = SSHClientInteraction(ssh, timeout=90, display=True)
interact.expect('admin:')

In [ ]:
#Store the output of the SSH commands
output = []
output.clear()

#Display list of trust certificates
interact.send("show cert list trust")
interact.expect('admin:')
output.append(interact.current_output)
certslist = interact.current_output

### 2. Inspect each certificate for validity

In [ ]:
#Extract the certifcate names using regex from the stored output
certnames = re.findall(r'\n(.*?)\:', certslist)
certnames.pop()

#Display certs before and after modification (esp. nonalphanumeric characters before '-')

df_cert_comp = pd.DataFrame(columns = ['Cert_Before','Cert_After'])
certcnt = 0
for i in range(len(certnames)):
    orig = certnames[i]
    repl = re.sub(r'[^a-zA-Z\d\s:]\-[^a-zA-Z\d\s:]','_', certnames[i])
    #certnames[i] = re.sub(r'[^a-zA-Z\d\s:]\-[^a-zA-Z\d\s:]','_', certnames[i])
    #certissue = re.sub(r'[^a-zA-Z\d\s:]\-[^a-zA-Z\d\s:]',' ', certissue)
    #print(certnames[i])
    df_cert_temp = pd.DataFrame({'Cert_Before' : [orig], 'Cert_After' : [repl]})
    df_cert_comp = df_cert_comp.append(df_cert_temp, ignore_index=True)
    certcnt+=1
print('Note: Number of trust certificates', certcnt)
df_cert_comp

In [ ]:
#Dataframe to create new columns matching the cert names
df_cert = pd.DataFrame(columns = ['Cert_Name', 'Valid_From', 'Valid_To', 'Validity'])

#Capture local time in UTC
lt = localtime()
ltimestamp = lt.replace(tzinfo=timezone.utc).timestamp()

#For each certificate, extract valid from, to and validate the certificate expiry
count = 0 
for i in range(len(certnames)):
    output.clear()
    #
    command = 'show cert trust ' + certnames[i]
    interact.send(command)
    interact.expect("Press <enter> for 1 line, <space> for one page, or <q> to quit",timeout = 5)
    interact.send("q")
    output.append(interact.current_output)
    interact.expect('admin:',timeout = 5)
    #interact.expect('Press <enter> for 1 line, <space> for one page, or <q> to quit')
    #output.append(interact.current_output)
    #interact.expect('admin:')
    #interact.send(" ")
    #output.append(interact.current_output)
    
    #Extract Validity From and To from the Certificate
    output_str = ' '.join(str(elem) for elem in output)
    valid_from = re.findall(r'From: (.*?)\n', output_str)
    valid_from_str = ' '.join(str(elem) for elem in valid_from)
    valid_to = re.findall(r'To:   (.*?)\n', output_str)
    valid_to_str = ' '.join(str(elem) for elem in valid_to)
    
    print(certnames[i],valid_from_str,valid_to_str)
    
    #Capture the Cert Name, Valid From and Valid To into a dataframe and insert the values
    df_cert_ind = pd.DataFrame({'Cert_Name' : [certnames[i]], 'Valid_From' : [valid_from_str], 'Valid_To' : [valid_to_str]})
    df_cert = df_cert.append(df_cert_ind, ignore_index=True)
    
    #Validate the timestamp for Valid To field
    if valid_to_str != "":
        dt = parsedate_to_datetime(valid_to_str)
        timestamp = dt.replace(tzinfo=timezone.utc).timestamp()
    else:
        valid_to_str = ltimestamp

    #Logic to validate the certificate expiry
    if timestamp > ltimestamp:
        df_cert.Validity[i] = "Valid"
    else:
        df_cert.Validity[i] = "Expired"
    
    count = count + 1

    time.sleep(10)

    print (df_cert)

In [ ]:
#Display all certificates with valid duration
df_cert

In [ ]:
#Subset and Display expired certificates
df_cert_exp = df_cert[df_cert.Validity == "Expired"]
df_cert_exp

### 3. Delete the expired certificates using the admin CLI

In [ ]:
# Delete the expired certificates
delcount = 0
for cert in df_cert_exp["Cert_Name"]:
    unit = cert.rpartition('/')[0]
    name = cert.rpartition('/')[2]
    if unit == "CallManager-trust":
        output.clear()
        interact.send(" ")
        interact.expect('admin:')
        command = "set cert delete CallManager" + name
        interact.send(command)
        time.sleep(5)
        interact.send("yes")
        interact.send(" ")
        interact.expect('admin:', timeout = 5)
    else if unit == "CAPF-trust":
        output.clear()
        interact.send(" ")
        interact.expect('admin:')
        command = "set cert delete CAPF" + name
        interact.send(command)
        time.sleep(5)
        interact.send("yes")
        interact.send(" ")
        interact.expect('admin:', timeout = 5)
    else if unit == "tomcat-trust":
        output.clear()
        interact.send(" ")
        interact.expect('admin:')
        command = "set cert delete tomcat" + name
        interact.send(command)
        time.sleep(5)
        interact.send("yes")
        interact.send(" ")
        interact.expect('admin:', timeout = 5)
    else if unit == "ipsec-trust":
        output.clear()
        interact.send(" ")
        interact.expect('admin:')
        command = "set cert delete ipsec" + name
        interact.send(command)
        time.sleep(5)
        interact.send("yes")
        interact.send(" ")
        interact.expect('admin:', timeout = 5)
    delcount = delcount + 1

    time.sleep(10)

print (delcount "certs deleted")

In [ ]:
#Display any certificates after deletion
df_cert_exp

### 4. Restart the tomcat service

In [ ]:
#Restart tomcat services
output.clear()
interact.send(" ")
interact.expect('admin:')
interact.send("utils service restart Cisco Tomcat")
interact.send(" ")
interact.expect('admin:')